#Importing The Module


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#Data Collection

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

#Data PreProcessing

In [ ]:
from google.colab import drive
import pandas as pd


df = pd.read_csv("/content/drive/MyDrive/abuse dataset_try_2/twitterdataset.csv")

# Replace 'Abuse' with 1 and 'Non_Abuse' with 0 in the 'label' column
df['label'] = df['label'].replace({'Abuse': 1, 'Non_Abuse': 0})

# Limit to 725 non-abuse samples only
non_abuse_samples = df[df['label'] == 0].head(2725)

# Change 0 to "Non_Abuse" in the label column
non_abuse_samples['label'] = "Non_Abuse"

# Save the modified dataset
non_abuse_samples.to_csv("/content/drive/MyDrive/abuse dataset_try_2/youtube_dataset_2725_non_abuse.csv", index=False)

print("✅ Dataset modified and saved as 'youtube_dataset_2725_non_abuse.csv'")


In [ ]:
# Load the two datasets
df1 = pd.read_csv("/content/drive/MyDrive/abuse_dataset_try/original_dataset _abuse _dataset.csv")
df2 = pd.read_csv("/content/drive/MyDrive/abuse dataset_try_2/youtube_dataset_2725_non_abuse.csv")

# Merge the datasets
merged_df = pd.concat([df1, df2], ignore_index=True)

# Save the merged dataset to a new CSV file
merged_df.to_csv("/content/drive/MyDrive/abuse dataset_try_2/merged_dataset.csv", index=False)

print("✅ Datasets merged and saved as 'merged_dataset.csv'")


In [ ]:
df = pd.read_csv(r'/content/drive/MyDrive/abuse dataset_try_2/merged_dataset.csv')

In [ ]:
df['label'].value_counts()

In [ ]:

df = df[['text', 'label' ]]
df.head()
df.shape


In [ ]:
df.shape

In [ ]:
df.duplicated().sum()

In [ ]:
df.drop_duplicates(inplace=True)
df.duplicated().sum()


In [ ]:
df.shape

In [ ]:
df.isnull().sum()


In [ ]:
df.shape

In [ ]:
df['label'].value_counts()


In [ ]:
df.columns

#Average word count

In [ ]:
df["word_count"] = df["text"].apply(lambda x: len(str(x).split()))

# Calculate the average word count
average_word_count = df["word_count"].mean()

print(f"The average word count is: {average_word_count}")

#Max word count

In [ ]:
# max word count
max_word_count = df['text'].max()
print(f"The maximum word count is: {max_word_count}")

In [ ]:
df['word_count'].describe()

In [ ]:
df.head()

#Use the Albert Model

In [ ]:
df = pd.read_csv(r'/content/drive/MyDrive/abuse dataset_try_2/merged_dataset.csv')

In [ ]:
from transformers import AlbertForSequenceClassification, AlbertTokenizer

# Load pre-trained ALBERT model for multiclass classification
model = AlbertForSequenceClassification.from_pretrained('albert-base-v1', num_labels=len(df['label'].unique()))  # Adjust num_labels as per your classes
tokenizer = AlbertTokenizer.from_pretrained("albert-base-v1")

# If you want to load a model from a specific checkpoint, you can do so by replacing the pretrained model name with the checkpoint path
# model = AlbertForSequenceClassification.from_pretrained('/content/results/checkpoint-1788')


#Split the Training,Validation

In [ ]:
from sklearn.model_selection import train_test_split
# Split the data into train and validation sets (80% train, 20% validation)
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
!pip install Datasets

In [ ]:
from datasets import Dataset
# Converting the DataFrames to Hugging Face datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

def preprocess_function(examples):
    examples['labels'] = examples['label']
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Tokenize the datasets
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

In [ ]:


# ✅ Step 5: Save to Disk (Optional, for future use)
val_dataset.save_to_disk("/content/drive/MyDrive/abuse dataset_try_2/val_dataset")

print("Validation dataset created successfully!")

In [ ]:
val_dataset

In [ ]:
df.columns

In [ ]:
from transformers import Trainer, TrainingArguments
from datasets import Dataset
import torch
from sklearn.metrics import accuracy_score

# ✅ Ensure labels are correctly formatted (Convert "Abuse" -> 1, "Non_Abuse" -> 0)
def convert_labels(example):
    if example['label'] == "Abuse":
        example['label'] = 1
    elif example['label'] == "Non_Abuse":
        example['label'] = 0
    return example

# ✅ Apply conversion to datasets
train_dataset = train_dataset.map(convert_labels)
val_dataset = val_dataset.map(convert_labels)

# ✅ Compute metrics function
def compute_metrics(p):
    preds = p.predictions.argmax(-1)  # Get predicted class
    labels = p.label_ids
    accuracy = accuracy_score(labels, preds)  # Compute accuracy
    return {"accuracy": accuracy}

# ✅ Define training arguments
training_args = TrainingArguments(
    output_dir='./results',              # Save model & checkpoints
    eval_strategy="epoch",               # Evaluate every epoch (fixed deprecation warning)
    save_strategy="epoch",               # Save checkpoint every epoch
    logging_dir='./logs',                # Log directory
    num_train_epochs=15,                  # Number of epochs
    per_device_train_batch_size=16,      # Batch size for training
    per_device_eval_batch_size=16,       # Batch size for evaluation
    logging_steps=10,                    # Log every 10 steps
    save_total_limit=3,                   # Keep last 3 checkpoints
    load_best_model_at_end=True,          # Load best model at the end
    report_to=[],                         # Disable WandB logging
)

# ✅ Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics  # Compute accuracy
)

# ✅ Start training
trainer.train()


#Evaluate the Result

In [ ]:
# Evaluate the model on validation data
val_results = trainer.evaluate(val_dataset)

# Extract loss and accuracy
val_loss = val_results.get("eval_loss", "N/A")
val_acc = val_results.get("eval_accuracy", 0)  # Default to 0 if not found

# Print validation loss and accuracy
print(f"Validation loss: {val_loss}")
print(f"Validation accuracy: {val_acc}")


#Save the Model

In [ ]:
trainer.save_model("/content/drive/MyDrive/abuse dataset_try_2/model")
tokenizer.save_pretrained("/content/drive/MyDrive/abuse dataset_try_2/model")


#Classification Report

In [ ]:
import numpy as np
import torch
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# Get predictions from the model
predictions = trainer.predict(val_dataset)

# Convert logits to predicted class labels
pred_labels = np.argmax(predictions.predictions, axis=1)

# Get the actual labels
true_labels = predictions.label_ids

# Print classification report
print("Classification Report:\n", classification_report(true_labels, pred_labels))


#Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(true_labels, pred_labels)

labels = ["Non-Abuse", "Abuse"]

# Plot the confusion matrix
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=labels, yticklabels=labels)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix (Abuse vs. Non-Abuse)")
plt.show()

#Load the Model

In [ ]:
from transformers import AlbertForSequenceClassification, Trainer, TrainingArguments

# Define model directory (change if different)
model_dir = "/content/drive/MyDrive/abuse dataset_try_2/model"

# Load fine-tuned ALBERT model
model = AlbertForSequenceClassification.from_pretrained(model_dir)

# Load training arguments (optional, for evaluation)
training_args = TrainingArguments(output_dir=model_dir)

# Reload trainer with fine-tuned model
trainer = Trainer(model=model, args=training_args)

print("✅ Model loaded successfully!")


In [ ]:
!pip install evaluate

# Plotting Training and Validation Accuracy

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from transformers import AlbertForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer
from datasets import load_from_disk
import evaluate

# Define paths
model_dir = "/content/drive/MyDrive/abuse dataset_try_2/model"
val_dataset_path = "/content/drive/MyDrive/abuse dataset_try_2/val_dataset"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("albert-base-v1")

# Load fine-tuned ALBERT model
model = AlbertForSequenceClassification.from_pretrained(model_dir)

# Load validation dataset
val_dataset = load_from_disk(val_dataset_path)

# Convert string labels to integers
label_map = {"Non_Abuse": 0, "Abuse": 1}  # Adjust mapping if needed

def convert_labels(example):
    example["label"] = label_map[example["label"]]  # Convert label string to integer
    return example

val_dataset = val_dataset.map(convert_labels)

# Ensure dataset is tokenized
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length")

val_dataset = val_dataset.map(tokenize_function, batched=True)

# ✅ Use Hugging Face's new `evaluate` library for accuracy
accuracy_metric = evaluate.load("accuracy")

# Function to compute accuracy
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)  # Convert logits to class predictions
    return accuracy_metric.compute(predictions=predictions, references=labels)

# Load training arguments (disable wandb)
training_args = TrainingArguments(
    output_dir=model_dir,
    report_to="none",  # Disables wandb logging
    per_device_eval_batch_size=16  # Adjust batch size if needed
)

# Reload trainer with accuracy metric
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics
)

# Evaluate on validation dataset
eval_results = trainer.evaluate(val_dataset)

# Extract loss and accuracy
val_loss = eval_results["eval_loss"]
val_accuracy = eval_results["eval_accuracy"]  # Now accuracy should be present!

# Extract training loss from logs
train_loss = [log["loss"] for log in trainer.state.log_history if "loss" in log]
epochs = list(range(1, len(train_loss) + 1))  # Assuming 1 epoch per entry

# Plot Training Loss vs Validation Loss
plt.figure(figsize=(10, 5))
plt.plot(epochs, train_loss, label="Training Loss", marker="o", linestyle="-")
plt.axhline(y=val_loss, color="r", linestyle="--", label="Validation Loss (Final)")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Training vs Validation Loss")
plt.legend()
plt.grid()
plt.show()

# Plot Validation Accuracy
plt.figure(figsize=(10, 5))
plt.axhline(y=val_accuracy, color="g", linestyle="--", label="Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Validation Accuracy")
plt.legend()
plt.grid()
plt.show()

print(f"✅ Model evaluated on validation set | Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}")


# Prediction for both Abuse and Non_Abuse

In [ ]:
from transformers import AlbertTokenizer, AlbertForSequenceClassification
import torch

# Load tokenizer and model (Ensure paths are correct)
model_path = "/content/drive/MyDrive/abuse dataset_try_2/model"
tokenizer = AlbertTokenizer.from_pretrained(model_path)
model = AlbertForSequenceClassification.from_pretrained(model_path)

# Move model to device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Example new texts
new_texts = [
    "Your ideas are brilliant and truly inspiring",
    "greece and serbia against all hd p youtube thank you brother greeks"
]

# Tokenize inputs (without return_tensors="pt")
tokenized_inputs = tokenizer(new_texts, padding="max_length", truncation=True, max_length=512)

# Convert tokenized inputs to PyTorch tensors manually
input_ids = torch.tensor(tokenized_inputs["input_ids"])
attention_mask = torch.tensor(tokenized_inputs["attention_mask"])

# Move inputs to the same device as the model
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)

# Make predictions
model.eval()
with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    predictions = torch.argmax(outputs.logits, dim=1)

# Print predictions
print("Predictions:", predictions.tolist())  # 0 = non-abusive, 1 = abusive


#Real Time Prediction

#Prediction for Abuse

In [ ]:
from transformers import AlbertTokenizer, AlbertForSequenceClassification
import torch

# Load tokenizer and model (Ensure paths are correct)
model_path = "/content/drive/MyDrive/abuse dataset_try_2/model"
tokenizer = AlbertTokenizer.from_pretrained(model_path)
model = AlbertForSequenceClassification.from_pretrained(model_path)

# Move model to device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Example abusive texts
abusive_texts = [
    "You are an idiot, no one cares about your opinion."

]

# Tokenize inputs
tokenized_inputs = tokenizer(abusive_texts, padding="max_length", truncation=True, max_length=512)

# Convert to PyTorch tensors
input_ids = torch.tensor(tokenized_inputs["input_ids"]).to(device)
attention_mask = torch.tensor(tokenized_inputs["attention_mask"]).to(device)

# Make predictions
model.eval()
with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    predictions = torch.argmax(outputs.logits, dim=1).tolist()

# Display only abusive sentences
print("\n🔴 Abusive Sentences:")
for text, label in zip(abusive_texts, predictions):
    if label == 1:
        print(f"- {text}")


#Prediction for Non abuse

In [ ]:
from transformers import AlbertTokenizer, AlbertForSequenceClassification
import torch

# Load tokenizer and model (Ensure paths are correct)
model_path = "/content/drive/MyDrive/abuse dataset_try_2/model"
tokenizer = AlbertTokenizer.from_pretrained(model_path)
model = AlbertForSequenceClassification.from_pretrained(model_path)

# Move model to device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Example non-abusive texts
non_abusive_texts = [
    "Your ideas are brilliant and truly inspiring",

]

# Tokenize inputs
tokenized_inputs = tokenizer(non_abusive_texts, padding="max_length", truncation=True, max_length=512)

# Convert to PyTorch tensors
input_ids = torch.tensor(tokenized_inputs["input_ids"]).to(device)
attention_mask = torch.tensor(tokenized_inputs["attention_mask"]).to(device)

# Make predictions
model.eval()
with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    predictions = torch.argmax(outputs.logits, dim=1).tolist()

# Display only non-abusive sentences
print("\n🟢 Non-Abusive Sentences:")
for text, label in zip(non_abusive_texts, predictions):
    if label == 0:
        print(f"- {text}")
